In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torchvision import datasets
import time

In [2]:
save_path = 'models/'
train_model = True

# Import data

In [3]:
cifar10_map = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}
data_path = '../dataset/cifar10'
mean = [0.4914, 0.4822, 0.4465] # copied from Q1
std = [0.2470, 0.2435, 0.2616] # copied from Q1
batch_size = 25

In [4]:
# Load the CIFAR-10 dataset
cifar10_train = data.DataLoader(
    datasets.CIFAR10(
        root=data_path,
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
    ),
    batch_size=batch_size,
    shuffle=True
)

Files already downloaded and verified


In [5]:
cifar10_test = data.DataLoader(
    datasets.CIFAR10(
        root=data_path,
        train=False,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
    ),
    batch_size=batch_size,
    shuffle=True
)

Files already downloaded and verified


# Models For Convolutional Neural Network 

In [6]:
n_channel_in = 3
n_channel_1  = 24
n_out = len(cifar10_map)

# Define the structure of the CNN
class ConvolutionalModel1(nn.Module):
    def __init__(self):
        super().__init__()
        # img is 32 x 32
        self.conv1 = nn.Conv2d(n_channel_in, n_channel_1, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        # img is 16 x 16
        
        # Add a fully connected layer at the end of the CNN
        self.fc = nn.Linear(n_channel_1 * 16 * 16, n_out)

    def forward(self, x):
        # Apply the convolutional filters, activation functions, and pooling layers
        out = self.pool1(self.act1(self.conv1(x)))
        
        # Flatten the output of the convolutional layers
        out = out.view(out.size(0), -1)
        # Apply the fully connected layer
        out = self.fc(out)
        return out


In [7]:
n_channel_in = 3
n_channel_1  = 16
n_channel_2  = 32
n_out = len(cifar10_map)

# Define the structure of the CNN
class ConvolutionalModel2(nn.Module):
    def __init__(self):
        super().__init__()
        # img is 32 x 32
        self.conv1 = nn.Conv2d(n_channel_in, n_channel_1, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        # img is 16 x 16
        self.conv2 = nn.Conv2d(n_channel_1, n_channel_2, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        # img is 8 x 8
        
        # Add a fully connected layer at the end of the CNN
        self.fc_nn = nn.Linear(n_channel_2 * 8 * 8, n_out)

    def forward(self, x):
        # Apply the convolutional filters, activation functions, and pooling layers
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        
        # Flatten the output of the convolutional layers
        out = out.view(out.size(0), -1)
        # Apply the fully connected layer
        out = self.fc_nn(out)
        return out


In [8]:
n_channel_in = 3
n_channel_1  = 16
n_channel_2  = 32
n_channel_3  = 64
n_out = len(cifar10_map)

# Define the structure of the CNN
class ConvolutionalModel3(nn.Module):
    def __init__(self):
        super().__init__()
        # img is 32 x 32
        self.conv1 = nn.Conv2d(n_channel_in, n_channel_1, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        # img is 16 x 16
        self.conv2 = nn.Conv2d(n_channel_1, n_channel_2, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        # img is 8 x 8
        self.conv3 = nn.Conv2d(n_channel_2, n_channel_3, kernel_size=3, padding=1)
        self.act3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2)
        # img is 4 x 4
        
        # Add a fully connected layer at the end of the CNN
        self.fc_nn = nn.Linear(n_channel_3 * 4 * 4, n_out)

    def forward(self, x):
        # Apply the convolutional filters, activation functions, and pooling layers
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = self.pool3(self.act3(self.conv3(out)))
        
        # Flatten the output of the convolutional layers
        out = out.view(out.size(0), -1)
        # Apply the fully connected layer
        out = self.fc_nn(out)
        return out


# Training function

In [9]:
def training(model, training_imgs, loss_fn, optimizer, n_epochs:int, report_period:int = 10):
    sum_time = 0
    for epoch in range(n_epochs):
        start = time.time()
        for images, labels in training_imgs:
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            outputs = model(images)
            # Compute the loss
            loss = loss_fn(outputs, labels)
            # Backward pass
            loss.backward()
            # Update the parameters
            optimizer.step()
            
        sum_time += (time.time() - start)
        if (epoch % report_period == 0) or (epoch == n_epochs - 1):
            print(f'epoch {epoch}:\t Loss: {loss:.15f}\t AvgTime: {sum_time/(epoch+1):.2f} s')

# 1 Layer Convolutional Neural Network
Question 2a

In [10]:
model = ConvolutionalModel1()
model 

ConvolutionalModel1(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=4096, out_features=10, bias=True)
)

In [11]:
learn_rate = 1e-2
report_rate = 2
n_epochs = 300

optimizer = optim.SGD(model.parameters(), lr=learn_rate)

loss = nn.CrossEntropyLoss()

In [12]:
if train_model:
    training(model, 
         cifar10_train, 
         loss, 
         optimizer, 
         n_epochs,
         report_period=report_rate
         )

epoch 0:	 Loss: 1.192782044410706	 AvgTime: 11.75 s
epoch 2:	 Loss: 1.342231035232544	 AvgTime: 12.28 s
epoch 4:	 Loss: 1.272854566574097	 AvgTime: 12.81 s
epoch 6:	 Loss: 0.996095418930054	 AvgTime: 12.92 s
epoch 8:	 Loss: 0.982859492301941	 AvgTime: 12.88 s
epoch 10:	 Loss: 0.798455834388733	 AvgTime: 12.92 s
epoch 12:	 Loss: 0.784306287765503	 AvgTime: 13.00 s
epoch 14:	 Loss: 0.763695597648621	 AvgTime: 12.98 s
epoch 16:	 Loss: 1.095210075378418	 AvgTime: 12.95 s
epoch 18:	 Loss: 0.838917553424835	 AvgTime: 12.96 s
epoch 20:	 Loss: 0.539276719093323	 AvgTime: 12.97 s
epoch 22:	 Loss: 0.716680228710175	 AvgTime: 12.99 s
epoch 24:	 Loss: 0.891482234001160	 AvgTime: 12.97 s
epoch 26:	 Loss: 0.703024268150330	 AvgTime: 12.97 s
epoch 28:	 Loss: 0.602555453777313	 AvgTime: 12.94 s
epoch 30:	 Loss: 0.442127764225006	 AvgTime: 12.92 s
epoch 32:	 Loss: 0.807938396930695	 AvgTime: 12.89 s
epoch 34:	 Loss: 0.452445536851883	 AvgTime: 12.91 s
epoch 36:	 Loss: 1.010540843009949	 AvgTime: 12.89 

In [13]:
if train_model :
    torch.save(model.state_dict(), save_path + '1_layer_cnn.pt')

In [14]:
load_model = ConvolutionalModel1()
load_model.load_state_dict(torch.load(save_path + '1_layer_cnn.pt'))

<All keys matched successfully>

In [15]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_train:
        outputs = load_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 50000
Accuracy: 0.85314


In [16]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_test:
        outputs = load_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 10000
Accuracy: 0.56970


# 2 Layer Convolutional Neural Network
Question 2b

In [17]:
model = ConvolutionalModel2()
model 

ConvolutionalModel2(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_nn): Linear(in_features=2048, out_features=10, bias=True)
)

In [18]:
learn_rate = 1e-2
report_rate = 2
n_epochs = 300

optimizer = optim.SGD(model.parameters(), lr=learn_rate)

loss = nn.CrossEntropyLoss()

In [19]:
if train_model:
    training(model, 
         cifar10_train, 
         loss, 
         optimizer, 
         n_epochs,
         report_period=report_rate
         )

epoch 0:	 Loss: 1.458693504333496	 AvgTime: 15.80 s
epoch 2:	 Loss: 1.171113610267639	 AvgTime: 15.92 s
epoch 4:	 Loss: 1.109544634819031	 AvgTime: 15.97 s
epoch 6:	 Loss: 0.877274453639984	 AvgTime: 15.87 s
epoch 8:	 Loss: 0.726249217987061	 AvgTime: 15.85 s
epoch 10:	 Loss: 0.808838188648224	 AvgTime: 15.86 s
epoch 12:	 Loss: 0.469166874885559	 AvgTime: 15.88 s
epoch 14:	 Loss: 0.930553793907166	 AvgTime: 15.89 s
epoch 16:	 Loss: 0.810217261314392	 AvgTime: 15.89 s
epoch 18:	 Loss: 0.706465780735016	 AvgTime: 15.90 s
epoch 20:	 Loss: 0.924698352813721	 AvgTime: 15.92 s
epoch 22:	 Loss: 0.771431982517242	 AvgTime: 15.95 s
epoch 24:	 Loss: 0.694605886936188	 AvgTime: 15.94 s
epoch 26:	 Loss: 0.762440264225006	 AvgTime: 15.96 s
epoch 28:	 Loss: 0.813989937305450	 AvgTime: 15.97 s
epoch 30:	 Loss: 0.335229635238647	 AvgTime: 15.98 s
epoch 32:	 Loss: 0.736242473125458	 AvgTime: 16.00 s
epoch 34:	 Loss: 0.547020316123962	 AvgTime: 15.99 s
epoch 36:	 Loss: 0.987408280372620	 AvgTime: 16.00 

In [20]:
if train_model :
    torch.save(model.state_dict(), save_path + '2_layer_cnn.pt')

In [21]:
load_model = ConvolutionalModel2()
load_model.load_state_dict(torch.load(save_path + '2_layer_cnn.pt'))

<All keys matched successfully>

In [22]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_train:
        outputs = load_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 50000
Accuracy: 0.89720


In [23]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_test:
        outputs = load_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 10000
Accuracy: 0.64760


# 3 Layer Convolutional Neural Network
extra

In [24]:
model = ConvolutionalModel3()
model 

ConvolutionalModel3(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_nn): Linear(in_features=1024, out_features=10, bias=True)
)

In [25]:
learn_rate = 1e-2
report_rate = 2
n_epochs = 300

optimizer = optim.SGD(model.parameters(), lr=learn_rate)

loss = nn.CrossEntropyLoss()

In [26]:
if train_model:
    training(model, 
         cifar10_train, 
         loss, 
         optimizer, 
         n_epochs,
         report_period=report_rate
         )

epoch 0:	 Loss: 1.280963778495789	 AvgTime: 16.19 s
epoch 2:	 Loss: 1.487491250038147	 AvgTime: 16.17 s
epoch 4:	 Loss: 1.011541962623596	 AvgTime: 16.17 s
epoch 6:	 Loss: 0.983987033367157	 AvgTime: 16.16 s
epoch 8:	 Loss: 0.667466819286346	 AvgTime: 16.15 s
epoch 10:	 Loss: 0.631892263889313	 AvgTime: 16.14 s
epoch 12:	 Loss: 0.677984178066254	 AvgTime: 16.14 s
epoch 14:	 Loss: 1.267391681671143	 AvgTime: 16.15 s
epoch 16:	 Loss: 0.406652420759201	 AvgTime: 16.14 s
epoch 18:	 Loss: 0.403346091508865	 AvgTime: 16.14 s
epoch 20:	 Loss: 0.508854806423187	 AvgTime: 16.13 s
epoch 22:	 Loss: 0.262133955955505	 AvgTime: 16.13 s
epoch 24:	 Loss: 0.637381792068481	 AvgTime: 16.13 s
epoch 26:	 Loss: 0.602623820304871	 AvgTime: 16.12 s
epoch 28:	 Loss: 0.178334668278694	 AvgTime: 16.12 s
epoch 30:	 Loss: 0.532237768173218	 AvgTime: 16.12 s
epoch 32:	 Loss: 0.419770091772079	 AvgTime: 16.12 s
epoch 34:	 Loss: 0.801614046096802	 AvgTime: 16.12 s
epoch 36:	 Loss: 0.210775047540665	 AvgTime: 16.12 

In [27]:
if train_model :
    torch.save(model.state_dict(), save_path + '3_layer_cnn.pt')

In [28]:
load_model = ConvolutionalModel3()
load_model.load_state_dict(torch.load(save_path + '3_layer_cnn.pt'))

<All keys matched successfully>

In [29]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_train:
        outputs = load_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 50000
Accuracy: 0.94010


In [30]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_test:
        outputs = load_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 10000
Accuracy: 0.67140
